# Task 2

Implement a random Metropolis-Hastings MCMC algorithm, based on the true likelihood (as computed by the exact method proposed by the author), to estimate the parameters of the model for the data mentioned in the paper.

In [ ]:
import sys
import os

# 1. On récupère le dossier où se trouve le notebook
current_dir = os.getcwd()

# 2. On remonte d'un cran pour trouver la racine du projet
# (Si votre notebook est dans un sous-sous-dossier, répétez os.path.dirname une fois de plus)
project_root = os.path.dirname(current_dir)

# 3. On ajoute ce chemin à Python s'il n'y est pas déjà
if project_root not in sys.path:
    sys.path.append(project_root)

# 4. Vérification (optionnel)
print(f"Racine ajoutée : {project_root}")
print(f"Dossier 'src' détecté ? : {os.path.isdir(os.path.join(project_root, 'src'))}")

In [ ]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
from scipy.special import gammaln
from scipy.stats import poisson, nbinom
from tqdm import tqdm
import openpyxl


# particles library usage 
import particles
import particles.state_space_models as ssm
import particles.distributions as dists


# project related packages
import src.cox_simulation as cx
import utils.plots as pl
import src.creal_filter as cf 
import src.particle_filter as pf
import utils.load_data as ld

In [ ]:
excel_path = os.path.join(project_root, "data", "data_groupe_T1000.xlsx")
y1, X1, h1, beta1, p1 = ld.load_data(excel_path, "Serie_1")
pl.plot_overlay_clean(y1, h1, T_show=1000, start=0)

In [ ]:
y2, X2, h2, beta2, p2 = ld.load_data(excel_path, "Serie_2")
pl.plot_overlay_clean(y2, h2, T_show=1000, start=0)


In [ ]:
y3, X3, h3, beta3, p3 = ld.load_data(excel_path, "Serie_3")
pl.plot_overlay_clean(y3, h3, T_show=1000, start=0)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed

# --- 1. FONCTIONS DE BASE ---

def log_prior(theta):
    """
    Définit les contraintes du modèle.
    theta = [phi, nu, c]
    """
    phi, nu, c = theta
    if 0 < phi < 0.999 and nu > 0 and c > 0:
        return 0.0
    else:
        return -np.inf

def run_metropolis_exact(y_data, exact_filter, n_iterations=5000, initial_theta=None, proposal_std=None, disable_tqdm=False):
    """
    Algorithme MCMC Random Walk Metropolis-Hastings.
    MODIFICATION : Retourne maintenant (chain, acceptance_rate)
    """
    if initial_theta is None:
        current_theta = np.array([0.5, 2.0, 0.5])
    else:
        current_theta = np.array(initial_theta)
        
    if proposal_std is None:
        proposal_std = np.array([0.02, 0.2, 0.1]) 
    
    chain = np.zeros((n_iterations, 3))
    accept_count = 0
    
    if log_prior(current_theta) == -np.inf:
        current_theta = np.array([0.5, 2.5, 0.5])
        
    current_log_prior = log_prior(current_theta)
    current_log_lik = exact_filter.log_likelihood(current_theta[0], current_theta[1], current_theta[2])
    current_log_post = current_log_lik + current_log_prior

    iterator = range(n_iterations)
    if not disable_tqdm:
        iterator = tqdm(iterator, desc="MCMC Sampling")
    
    for i in iterator:
        proposal = current_theta + np.random.normal(0, proposal_std)
        prop_log_prior = log_prior(proposal)
        
        if prop_log_prior == -np.inf:
            chain[i] = current_theta
        else:
            try:
                prop_log_lik = exact_filter.log_likelihood(proposal[0], proposal[1], proposal[2])
                prop_log_post = prop_log_lik + prop_log_prior
                
                log_alpha = prop_log_post - current_log_post
                
                if np.log(np.random.rand()) < log_alpha:
                    current_theta = proposal
                    current_log_post = prop_log_post
                    accept_count += 1
            except Exception:
                pass
        
        chain[i] = current_theta

    # CALCUL DU TAUX
    acc_rate = accept_count / n_iterations
    
    # On affiche seulement si on n'est pas en mode "silencieux" (parallèle)
    if not disable_tqdm:
        print(f"Taux d'acceptation final : {acc_rate:.2%}")
        
    # RETOURNE UN TUPLE MAINTENANT
    return chain, acc_rate

# --- 2. FONCTION PARALLÈLE (LE WORKER) ---

def _worker_chain(seed, y, exact_filter, n_iter, proposal_std):
    np.random.seed(seed)
    
    start_phi = np.random.uniform(0.5, 0.95) # Attention à 0.999 c'est risqué
    start_nu  = np.random.uniform(1.5, 3.5)
    start_c   = np.random.uniform(0.1, 0.3) # Eviter 0 pile
    start_theta = [start_phi, start_nu, start_c]
    
    # Le worker renvoie (chain, rate)
    return run_metropolis_exact(
        y_data=y,
        exact_filter=exact_filter,
        n_iterations=n_iter,
        initial_theta=start_theta,
        proposal_std=proposal_std,
        disable_tqdm=True 
    )

# --- 3. ORCHESTRATEUR MULTI-CHAÎNES ---

def run_multi_chain_mcmc(y, exact_filter, n_chains=4, n_iter=2000, proposal_std=[0.008, 0.07, 0.03], burn_in=500, true_params=None):
    print(f"🚀 Lancement de {n_chains} chaînes MCMC en parallèle sur CPU...")
    
    # Exécution parallèle
    results = Parallel(n_jobs=-1)(
        delayed(_worker_chain)(
            seed=k, 
            y=y, 
            exact_filter=exact_filter, 
            n_iter=n_iter, 
            proposal_std=proposal_std
        ) for k in tqdm(range(n_chains), desc="Progression globale")
    )
    
    # DÉCOMPOSITION DES RÉSULTATS
    # results est une liste de tuples [(chain1, rate1), (chain2, rate2), ...]
    chains = np.array([r[0] for r in results]) # On extrait juste les chaînes pour numpy
    rates = [r[1] for r in results]           # On extrait les taux
    
    # AFFICHAGE DES TAUX PROPREMENT
    print("\n--- TAUX D'ACCEPTATION PAR CHAÎNE ---")
    for k, r in enumerate(rates):
        status = "✅" if 0.2 <= r <= 0.5 else "⚠️"
        print(f"Chaîne {k+1}: {r:.2%} {status}")
    print(f"Moyenne globale: {np.mean(rates):.2%}")

    # --- VISUALISATION ---
    print("\n✅ Génération des graphiques...")
    param_names = [r'$\phi$', r'$\nu$', r'$c$']
    fig, axes = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
    colors = plt.cm.jet(np.linspace(0, 1, n_chains))
    
    for i in range(3):
        ax = axes[i]
        for k in range(n_chains):
            ax.plot(chains[k][:, i], alpha=0.5, color=colors[k], lw=1)
        
        if true_params is not None:
            ax.axhline(true_params[i], color='black', linestyle='--', linewidth=2, label='Vrai')
            
        ax.set_ylabel(param_names[i])
        ax.set_title(f"Traceplot : {param_names[i]}")
        if i == 0 and true_params is not None: ax.legend(loc='upper right')
        ax.grid(True, alpha=0.3)
        
    plt.xlabel("Itérations")
    plt.tight_layout()
    plt.show()
    
    # --- STATISTIQUES GLOBALES ---
    all_samples = np.vstack([c[burn_in:] for c in chains])
    global_mean = all_samples.mean(axis=0)
    global_std = all_samples.std(axis=0)
    
    print("\n--- RÉSULTATS FINAUX (Agrégés) ---")
    print(f"Phi : {global_mean[0]:.4f} +/- {global_std[0]:.4f}")
    print(f"Nu  : {global_mean[1]:.4f} +/- {global_std[1]:.4f}")
    print(f"c   : {global_mean[2]:.4f} +/- {global_std[2]:.4f}")
    
    return chains

# --- UTILISATION ---
res = run_multi_chain_mcmc(y, exact, n_chains=10, n_iter=1000, true_params=[0.98, 2.5, 0.2])